#Preparation
Usage:[My github page](https://github.com/older4/Spleeter_easy_colab)  

First, we will install necessaly libraries in google colab machine.
When you run a cell, be sure to click [ ] object or press "ctl + enter". 

In [ ]:
!pip install pydub
!pip install spleeter-gpu

**Don't forget to press 'RESTART RUNTIME' button**

# Run Spleeter 
When you face bugs, try to restart a runtime or wait a minutes and restart its cell.

In [ ]:
#@title Load modules
#@markdown **Whenever after you restart a runtime, you must run this cell.**

import warnings
warnings.simplefilter('ignore')

from IPython.display import Audio,display,clear_output,update_display
from google.colab import files

import glob
import os
import subprocess
import re
import ipywidgets as widgets
import shutil

from pydub import AudioSegment 
from spleeter.separator import Separator

MP3_BITRATE = "192k"#@param {type:"string"}

# file operation
def get_uploaded_music_path_list():
  upload_music_path_list = [simplify_filename(path) for path in glob.glob('/content/*') 
                            if re.search(r".(wav|mp3|ogg|m4a|wma|flac)", path.lower())]
  if(upload_music_path_list):
    return upload_music_path_list
  else:
    raise ValueError("Music File not found. Please upload some music.")

def get_uploaded_music_name_list():
  return [os.path.basename(p) for p in get_uploaded_music_path_list()]

def get_output_folder_path_list():
  output_list_folder_path_list = glob.glob("/content/audio_output/*")
  if(output_list_folder_path_list):
    return output_list_folder_path_list
  else:
    raise ValueError("Unmix file not found.")

def get_music_name(music_path):
  return os.path.basename(music_path).split(".")[0]

def simplify_filename(path):
  file_name = get_music_name(path)
  ext = os.path.basename(path).split(".")[1]
  renamed_file = re.sub(r"[-()\"#/@;:<>{}`+=~|!?,　]", "", file_name)[:20] + "." +ext
  new_file_path = os.path.dirname(path) + "/" + renamed_file
  os.rename(path,new_file_path)
  return new_file_path

# spleeter operation
def run_separation(target_music,part_mode):
  music_name = get_music_name(target_music)
  tmp_folder = "/content/audio_output/tmp/"
  output_folder ="/content/audio_output/{name}/".format(name=music_name)
  output_part_folder ="/content/audio_output/{name}/{mode}part/".format(name=music_name,mode=part_mode)

  separator = Separator('spleeter:{mode}stems'.format(mode=part_mode))
  separator.separate_to_file(target_music, tmp_folder)
  separated_folder = tmp_folder + music_name + "/"

  if(os.path.exists(output_part_folder)):
    shutil.rmtree(output_part_folder)
  os.makedirs(output_folder, exist_ok=True)
  shutil.move(separated_folder,output_part_folder)

  # convert wav to mp3
  os.makedirs(output_part_folder[:-1]+"_mp3/",exist_ok=True)

  for path in glob.glob(output_part_folder+"*.wav"):
    part_name = os.path.basename(path).split(".")[0]
    AudioSegment.from_wav(path).export(output_part_folder[:-1]+"_mp3/"+part_name+".mp3",format="mp3",bitrate=MP3_BITRATE)

# misc operation
def get_part_file_or_make(music_path,part,part_mode,encode):
  music_name = get_music_name(music_path)
  if(encode == "mp3"):
    target_file ="/content/audio_output/{name}/{mode}part_mp3/{part}.mp3".format(name=music_name,mode=part_mode,part=part)
  else:
    target_file ="/content/audio_output/{name}/{mode}part/{part}.wav".format(name=music_name,mode=part_mode,part=part)

  if(os.path.exists(target_file)):
    return target_file
  else:
    run_separation(music_path,part_mode)
    return target_file

# combine operation
def get_combinetion_file_or_combine(target_music,parts,encode,is_piano):
  music_name = get_music_name(target_music)
  mix_output_folder ="/content/audio_output/{name}/mix/".format(name=music_name)
  sounds_list = []
  mixed_name = music_name
  
  if(is_piano):
    part_mode = 5
  else:
    part_mode = 4

  for part in parts:
    mixed_name += "_" +part
    sounds_list.append(AudioSegment.from_wav(
        get_part_file_or_make(target_music,part,part_mode,"wav")))

  destination_file = mix_output_folder+mixed_name+"."+encode

  if(os.path.exists(destination_file)):
    return destination_file

  else:
    mixed = AudioSegment.silent(duration=len(sounds_list[0]))
    for sound in sounds_list:
      mixed = mixed.overlay(sound)
    
    os.makedirs(mix_output_folder,exist_ok=True)
    mixed.export(mix_output_folder+mixed_name+".wav",format= "wav")
    mixed.export(mix_output_folder+mixed_name+".mp3",format = "mp3" ,bitrate=MP3_BITRATE)

    return destination_file

def combine_parts(target_music,part_dic,encode = "mp3"):
  music_name = get_music_name(target_music)
  mix_output_folder ="/content/audio_output/{name}/mix/".format(name=music_name)
  # extract checked(value == 1) keys
  parts = [k for k, v in part_dic.items() if v == 1]
  print(parts)

  # exception patterns not need combination--------------------------
  if(len(parts)==0):
    return None

  elif(len(parts)==1 and "vocals" in parts):
    print("vocal only")
    return get_part_file_or_make(target_music,"vocals",2,encode)
  
  elif(len(parts)==1):
    return get_part_file_or_make(target_music,parts[0],5,encode)

  elif(len(parts)==4 and not("vocals" in parts)):
    print("karaoke")
    return get_part_file_or_make(target_music,"accompaniment",2,encode)

  # ----------------------------------------------------------------
  if("other" in parts and "piano" in parts):
    print("4 stem mode")
    parts.remove("piano")
    return get_combinetion_file_or_combine(target_music,parts,encode,is_piano=False)

  else:
    print("in piano mode (5 stem)")
    return get_combinetion_file_or_combine(target_music,parts,encode,is_piano=True)
  

In [ ]:
#@title Upload your music file
#@markdown support file: wav, mp3, ogg, m4a, wma, flac  
uploaded = files.upload()

In [ ]:
#@title Select music from Youtube URL
import youtube_dl
ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl':  "%(title)s" + '.%(ext)s',
    'postprocessors': [
        {'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
         'preferredquality': '192'},
        {'key': 'FFmpegMetadata'},
    ],
}
ydl = youtube_dl.YoutubeDL(ydl_opts)
Youtube_url = "https://www.youtube.com/watch?v=sr--GVIoluU"#@param {type:"string"}
info_dict = ydl.extract_info(Youtube_url, download=True)


In [ ]:
#@title Listen split (unmix) music on browther 

class try_listen:
  def __init__(self):
    self.select_mode = widgets.Dropdown(
    options=[('5 stem', 5), ('4 stem', 4), ('2 stem', 2)],
    description='Stem mode:',
    )
    self.select_music = widgets.Dropdown(
      options=get_uploaded_music_name_list(),
      description='Music list:',
    )
    self.listen_button = widgets.Button(
    description='Listing',
    tooltip='start listing',
    icon='check'
    )

    self.listen_music_selecter = widgets.AppLayout(
      header=None,
      left_sidebar=self.select_music,
      center=self.select_mode,
      right_sidebar=self.listen_button,
      footer=None               
    )
    self.output = widgets.Output(layout={'border': '1px solid black'})
    self.listen_button.on_click(self.listen_button_clicked)
    display(self.listen_music_selecter)
    display(self.output)

  def set_button_disabled(self,mode):
    self.listen_button.disabled=mode
    self.select_music.disabled=mode
    self.select_mode.disabled=mode

  def listen_button_clicked(self,b):
    PART5 = ["vocals","piano","other","drums","bass"]
    PART4 = ["vocals","others","drums","bass"]
    PART2 = ["vocals","accompaniment"]

    self.set_button_disabled(True)
    
    target_path = "/content/"+self.select_music.value

    if(self.select_mode.value == 5):
      parts = PART5
    elif(self.select_mode.value == 4):
      parts = PART4
    elif(self.select_mode.value == 2):
      parts = PART2

    for p in parts:
      destination_file = get_part_file_or_make(
          target_path,p,self.select_mode.value,"mp3"
      )
      print(self.select_music.value+": "+p)
      display(Audio(destination_file))
    
    self.set_button_disabled(False)

a = try_listen()


In [ ]:
#@title Download split (unmix) music
#@markdown It takes a while, wait a minutes. When you chose a wav type, its size is big and takes a long time.


class parts_downloader:
  def __init__(self):
    self.select_music = widgets.Dropdown(
      options=get_uploaded_music_name_list(),
      description='Music:',
    )
    self.select_mode = widgets.Dropdown(
      options=[('5 stem', 5), ('4 stem', 4), ('2 stem', 2)],
      description='Stem mode:',
    )

    self.select_quality = widgets.Dropdown(
        options=["mp3","wav"],
        description='type:'
    )

    self.select_zip = widgets.Dropdown(
        options=[("zip",1),("one by one",0)],
        description='How to:'
    )

    self.download_button = widgets.Button(
      description='Download',
      tooltip='start download',
    )
    self.download_music_selecter = widgets.TwoByTwoLayout(
      top_left=self.select_music,
      top_right=self.select_mode,
      bottom_left=self.select_quality,
      bottom_right=self.select_zip               
    )
    self.download_out = widgets.Output(layout={'border': '1px solid black'})
    self.download_button.on_click(self.download_button_clicked)
    
    display(self.download_music_selecter)
    display(self.download_button)
    display(self.download_out)

  def download_button_clicked(self,b):
    PART5 = ["vocals","piano","other","drums","bass"]
    PART4 = ["vocals","others","drums","bass"]
    PART2 = ["vocals","accompaniment"]
    self.download_button.disabled=True
    self.select_music.disabled=True
    self.select_mode.disabled=True
    self.select_zip.disabled = True
    self.select_quality.disabled = True
    
    target_path = "/content/"+self.select_music.value
    music_name = get_music_name(self.select_music.value)


    print(music_name+":"+str(self.select_mode.value)+"パート")
    
    if(self.select_zip.value):
      if(self.select_quality.value == "mp3"):
        download_file_path = 'audio_output/{name}/{mode}part_mp3/'.format(name=music_name,mode=self.select_mode.value)
      elif(self.select_quality.value == "wav"):
        download_file_path = 'audio_output/{name}/{mode}/{name}'.format(name=music_name,mode=self.select_mode.value)
      
      zip_name =music_name+"_"+"part"+str(self.select_mode.value)+"_"+self.select_quality.value
      zip_folder = "/content/zip/{name}/".format(name = music_name)

      try:
        print("Check file.")
        files.download(zip_folder+zip_name+".zip")
      except FileNotFoundError:
        get_part_file_or_make(
            target_path,"vocals",self.select_mode.value,self.select_quality.value
        )
        print("zipping.")
        shutil.make_archive(zip_folder+zip_name,'zip',root_dir=download_file_path)
        print("Done, and start download.") 
        files.download(zip_folder+zip_name+".zip")
        
    else:
      if(self.select_mode.value == 5):
        parts = PART5
      elif(self.select_mode.value == 4):
        parts = PART4
      elif(self.select_mode.value == 2):
        parts = PART2
      for p in parts:
        destination_file = get_part_file_or_make(
            target_path,p,self.select_mode.value,self.select_quality.value
        )
        files.download(destination_file)
    
    self.download_button.disabled=False
    self.select_music.disabled=False
    self.select_mode.disabled=False
    self.select_zip.disabled = False
    self.select_quality.disabled = False

a = parts_downloader()

In [ ]:
#@title Synth unmix part
#@markdown You can get karaoke or synthesized music with parts you want. 


class mixer:
  def __init__(self):
    self.select_music = widgets.Dropdown(
      options=get_uploaded_music_name_list(),
      description='Music:',
    )

    self.select_quality = widgets.Dropdown(
        options=["mp3","wav"],
        description='type:'
    )

    self.mix_download_button = widgets.Button(
      description='Download',
      tooltip='start download',
    )

    self.mix_listen_button = widgets.Button(
      description='Listening',
      tooltip='start listenig without download.',
      icon='check'
    )

    self.synth_selecter = widgets.TwoByTwoLayout(
      top_left=self.select_music,
      top_right=self.select_quality,
      bottom_left=self.mix_listen_button,
      bottom_right=self.mix_download_button               
    )

    self.vocal_check =widgets.Checkbox(
    value=False,
    description='vocal',
    disabled=False
    )
    self.others_check =widgets.Checkbox(
        value=False,
        description='other(guitar etc...)',
        disabled=False
    )
    self.piano_check =widgets.Checkbox(
        value=False,
        description='piano',
        disabled=False
    )
    self.bass_check =widgets.Checkbox(
        value=False,
        description='bass',
        disabled=False
    )
    self.drum_check =widgets.Checkbox(
        value=False,
        description='drum',
        disabled=False
    )

    self.check_boxs = widgets.GridBox(
        (self.vocal_check,self.others_check,self.piano_check,self.bass_check,self.drum_check),
        layout=widgets.Layout(grid_template_columns="repeat(3, 200px)"))

    self.mix_out = widgets.Output(layout={'border': '1px solid black'})
    self.mix_download_button.on_click(self.mix_download_button_clicked)
    self.mix_listen_button.on_click(self.mix_listen_button_clicked)
    
    display(self.synth_selecter)
    display(self.check_boxs)
    display(self.mix_out)

  def set_button_disabled(self,mode):
    self.mix_listen_button.disabled = mode
    self.mix_download_button.disabled=mode
    self.select_music.disabled=mode
    self.select_quality.disabled = mode

  def get_combination_parts(self):
    selected_part_dic = {"vocals":0,"other":0,"bass":0,"piano":0,"drums":0}
    if(self.vocal_check.value):
      selected_part_dic["vocals"]=1
    if(self.others_check.value):
      selected_part_dic["other"]=1
    if(self.piano_check.value):
      selected_part_dic["piano"]=1
    if(self.bass_check.value):
      selected_part_dic["bass"]=1
    if(self.drum_check.value):
      selected_part_dic["drums"]=1

    if(not 1 in selected_part_dic.values()):
      print("At least select one option!")
      return False

    target_path = "/content/"+self.select_music.value
    return combine_parts(target_path,selected_part_dic,self.select_quality.value)

  def mix_download_button_clicked(self,b):
    self.set_button_disabled(True)
    path = self.get_combination_parts()
    if(path):
      files.download(path)
    self.set_button_disabled(False)
  
  def mix_listen_button_clicked(self,b):
    self.set_button_disabled(True)
    path = self.get_combination_parts()
    if(path):
      print(self.select_music.value +":"+ os.path.basename(path))
      display(Audio(path))
    self.set_button_disabled(False)

a = mixer()